# Clustering Scifi Stories
I scraped these from Tor.com

In [ ]:
import scrapy
import scrapy.crawler
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess({'AUTOTHROTTLE_ENABLED': True, # or download delay
                          'HTTPCACHE_ENABLED': True, # remove for final scrape to get live data
                          'ROBOTSTXT_OBEY': True, 
                          'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)' })
class TorSpider(scrapy.Spider):
    name = 'tors'
    start_urls = ['https://www.tor.com/category/all-fiction/original-fiction/']
    def parse(self, response):
        for item in response.xpath('//*/header/h2/a'):
            print(item)
            yield scrapy.Request(item, callback=story)

    def story(self, response):
        yield {'Title': response.xpath('//*/header/h2[2]/a.text()').extract(),
               'Author': response.xpath('//*/header/a.text()').extract(),
               'Body': response.xpath('//*/div').extract()}


process.crawl(TorSpider)
process.start()